<a href="https://colab.research.google.com/github/SpellOnYou/CLab21/blob/main/dummymodule/batch_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random, torch, math
!pip install -q ipdb
import ipdb

     |████████████████████████████████| 788kB 7.2MB/s 
     |████████████████████████████████| 368kB 11.1MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.23.1 which is incompatible.


In [2]:
class Linear():
    def __init__(self, w, b):
        self.w, self.b = w, b
    def forward(self, x): 
        self.inp = x
        self.out = self.inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [3]:
class Relu():
    def forward(self, x):
        self.inp = x
        self.out = x.clamp_min(0.) - 0.5
        return self.out

    def backward(self):
        self.inp.g = self.out.g* (self.inp>0).float()

In [38]:
class Softmax():
    def forward(self, x):
        self.inp = x
        self.out = (x.exp()/ x.exp().sum(-1, keepdim=True))
        return self.out

    def backward(self):
        self.inp.g = (1-self.out.g)*self.out.g

class CrossEntropy():
    def __call__(self, y, yhat):
        self.yhat = yhat
        self.y = y
        return (-yhat.log()[range(yhat.shape[0]), y.max(dim=1).indices]).mean()
    
    def backward(self):
        self.yhat.g = self.yhat - self.y

In [40]:
class DummyModel():
    def __init__(self, w1, b1, w2, b2):
        self.loss = CrossEntropy()
        self.layers = [Linear(w1,b1), Relu(), Linear(w2, b2), Softmax()]
    
    def forward(self, x, y):
        for layer in self.layers:
            # ipdb.set_trace()
            x = layer.forward(x)
        self.out = x
        return self.out

    def backward(self):
        self.loss.backward()
        for layer in reversed(self.layers):
            layer.backward()

In [41]:
m, n, h, c = 5000, 100, 50, 6
w1 = torch.randn(m, h) / math.sqrt(h)
w2 = torch.randn(h, c) / math.sqrt(c)
b1 = torch.randn(h)
b2 = torch.randn(c)

In [42]:
dummy_x, dummy_y = init_data()

In [43]:
model = DummyModel(w1, b1, w2, b2)

In [44]:
out = model.forward(dummy_x, dummy_y)

In [45]:
loss = model.loss(dummy_y, out)

In [46]:
model.backward()

In [55]:
[hasattr(l, 'w') for l in model.layers]

[True, False, True, False]

## Training process

In [82]:
def init_data():
    x = torch.randn(n, m)
    y = torch.zeros(n, c)
    y[range(n), random.choices(range(c), k=n)] =1    
    return x, y

In [87]:
n, m, h, c = 50000, 100, 50, 6
w1 = torch.randn(m, h) / math.sqrt(h)
w2 = torch.randn(h, c) / math.sqrt(c)
b1 = torch.randn(h)
b2 = torch.randn(c)

In [88]:
dummy_x, dummy_y = init_data()

In [89]:
model = DummyModel(w1, b1, w2, b2)

In [90]:
res = model.forward(dummy_x, dummy_y)
model.loss(dummy_y, res)

tensor(4.2179)

In [91]:
def train(epochs, bs, lr):
    for e in range(epochs):
        for bs_i in range((n-1)//bs + 1):
            str_idx, end_idx = bs_i*bs, (bs_i+1)*bs
            x_batch, y_batch = dummy_x[str_idx:end_idx], dummy_y[str_idx:end_idx]
            res = model.forward(x_batch, y_batch)
            loss = model.loss(y_batch, res)
            model.backward()
            with torch.no_grad():
                for layer in model.layers:
                    if hasattr(layer, 'w'): #if they have parameter attribute
                        layer.w -= layer.w.g * lr
                        layer.b -= layer.b.g * lr
                        layer.w.g.zero_() #initialize them to zero
                        layer.b.g.zero_()

In [92]:
train(1, 32, 0.1)

In [93]:
res = model.forward(dummy_x, dummy_y)
model.loss(dummy_y, res)

tensor(nan)